# Proposal for Deep Learning Approach to Network Intrusion Detection:

### Evaluation of Generative Architecture Models vs Discriminative Architecture Models

Basic DNN

By-Cliff Green

In [153]:
# Here are some imports that are used along this notebook
import math
import itertools
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from collections import OrderedDict
import glob
import tensorflow as tf

In [309]:
train20_nsl_kdd_dataset_path = "NSL-KDD-Dataset-master/NSL-KDD-Dataset-master/KDDTrain+_20Percent.txt"
train_nsl_kdd_dataset_path = "NSL-KDD-Dataset-master/NSL-KDD-Dataset-master/KDDTrain+.txt"
test_nsl_kdd_dataset_path = "NSL-KDD-Dataset-master/NSL-KDD-Dataset-master/KDDTest-21.txt"


col_names = np.array(["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels","attrib43"])


attack_dict = {
    'normal': 'normal',
    
    'back': 'DoS',
    'land': 'DoS',
    'neptune': 'DoS',
    'pod': 'DoS',
    'smurf': 'DoS',
    'teardrop': 'DoS',
    'mailbomb': 'DoS',
    'apache2': 'DoS',
    'processtable': 'DoS',
    'udpstorm': 'DoS',
    
    'ipsweep': 'Probe',
    'nmap': 'Probe',
    'portsweep': 'Probe',
    'satan': 'Probe',
    'mscan': 'Probe',
    'saint': 'Probe',

    'ftp_write': 'R2L',
    'guess_passwd': 'R2L',
    'imap': 'R2L',
    'multihop': 'R2L',
    'phf': 'R2L',
    'spy': 'R2L',
    'warezclient': 'R2L',
    'warezmaster': 'R2L',
    'sendmail': 'R2L',
    'named': 'R2L',
    'snmpgetattack': 'R2L',
    'snmpguess': 'R2L',
    'xlock': 'R2L',
    'xsnoop': 'R2L',
    'worm': 'R2L',
    
    'buffer_overflow': 'U2R',
    'loadmodule': 'U2R',
    'perl': 'U2R',
    'rootkit': 'U2R',
    'httptunnel': 'U2R',
    'ps': 'U2R',    
    'sqlattack': 'U2R',
    'xterm': 'U2R'
}


# In[3]:


def _label2(x):
    if x['labels'] == 'normal':
        return 'normal'
    else:
        return 'attack'

def returnvalue(x):
    return attack_dict.get(x['labels'])


df_kdd_dataset_train = pd.read_csv(train_nsl_kdd_dataset_path, index_col=None, header=0, names=col_names)
df_kdd_dataset_test = pd.read_csv(test_nsl_kdd_dataset_path, index_col=None, header=0, names=col_names)

df_kdd_dataset_train['label2'] = df_kdd_dataset_train.apply(_label2,axis=1)
df_kdd_dataset_train['label3'] = df_kdd_dataset_train.apply(returnvalue,axis=1)

df_kdd_dataset_test['label2'] = df_kdd_dataset_test.apply(_label2,axis=1)
df_kdd_dataset_test['label3'] = df_kdd_dataset_test.apply(returnvalue,axis=1)

In [349]:
#df_kdd_dataset_train = df_kdd_dataset_train.dropna()

In [350]:
#df_kdd_dataset_train.isnull().values.any()

In [351]:
#df_kdd_dataset_train['protocol_type'].unique()

In [352]:
#col_names[binary_inx]

In [189]:
nominal_inx = [1, 2, 3]
binary_inx = [6, 11, 13, 14, 20, 21]
numeric_inx = list(set(range(41)).difference(nominal_inx).difference(binary_inx))

In [211]:
numeric_inx = [0,
 4,
 5,
 7,
 8,
 9,
 10,
 12,
 15,
 16,
 17,
 18,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40]

In [212]:
catagorical_inx = [1, 2, 3, 6, 11, 13, 14, 20, 21]

In [213]:
cols_to_norm = col_names[numeric_inx]

In [310]:
df_kdd_dataset_train[cols_to_norm] = df_kdd_dataset_train[cols_to_norm].apply(lambda x: (x-x.min())/(x.max()-x.min() ))
df_kdd_dataset_test[cols_to_norm] = df_kdd_dataset_test[cols_to_norm].apply(lambda x: (x-x.min())/(x.max()-x.min() ))


In [215]:
import tensorflow as tf

In [216]:
Numeric = []
for i in numeric_inx+binary_inx:
    #print(i)
    i = tf.feature_column.numeric_column(col_names[i])
    Numeric.append(i)

In [217]:
len(df_kdd_dataset_train[col_names[1]].unique())

3

In [218]:
Nominal = []
for i in nominal_inx: 
    i = tf.feature_column.categorical_column_with_hash_bucket(col_names[i],hash_bucket_size=len(df_kdd_dataset_train[col_names[i]].unique()))
    Nominal.append(i)

In [219]:
#assign_group = tf.feature_column.categorical_column_with_vocabulary_list('Group', ['A', 'B', 'C', 'D'])

In [220]:
#assign_group = tf.feature_column.categorical_column_with_hash_bucket('Group', hash_bucket_size=10)

In [221]:
#making buckets for age varaible
#age_bucket = tf.feature_column.bucketized_column(age, boundaries=[20,30,40,50,60,70,80])

In [222]:
feat_cols = Nominal+Numeric

In [311]:
X_train = df_kdd_dataset_train.drop(['labels','attrib43','label2','label3','num_outbound_cmds'], axis = 1)

X_test = df_kdd_dataset_test.drop(['labels','attrib43','label2','label3','num_outbound_cmds'], axis = 1)

In [312]:
y_train = df_kdd_dataset_train['label3']

y_test = df_kdd_dataset_test['label3']


In [313]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['normal', 'DoS','Probe', 'R2L','U2R'])
y_train = le.transform(y_train)
y_test = le.transform(y_test)


In [314]:
y_test = pd.DataFrame(y_test)
y_train = pd.DataFrame(y_train)

In [227]:
#from sklearn.model_selection import train_test_split

In [228]:
#X_train, X_test, y_train, y_test = train_test_split(x_data, labels,  test_size = .3, random_state = 101)

In [233]:
X_train.shape

(17633, 40)

In [315]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100, num_epochs=1000, shuffle=True)

## Shallow Net

In [316]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols, n_classes=6)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\Green\\AppData\\Local\\Temp\\tmpnritn_33', '_session_config': None}


In [317]:
model.train(input_fn=input_func, steps = 1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Green\AppData\Local\Temp\tmpnritn_33\model.ckpt.
INFO:tensorflow:step = 1, loss = 179.176
INFO:tensorflow:global_step/sec: 172.841
INFO:tensorflow:step = 101, loss = 13.5142 (0.580 sec)
INFO:tensorflow:global_step/sec: 244.464
INFO:tensorflow:step = 201, loss = 21.7895 (0.410 sec)
INFO:tensorflow:global_step/sec: 212.652
INFO:tensorflow:step = 301, loss = 8.89953 (0.470 sec)
INFO:tensorflow:global_step/sec: 185.37
INFO:tensorflow:step = 401, loss = 5.72815 (0.538 sec)
INFO:tensorflow:global_step/sec: 246.884
INFO:tensorflow:step = 501, loss = 6.27231 (0.405 sec)
INFO:tensorflow:global_step/sec: 232.459
INFO:tensorflow:step = 601, loss = 8.68018 (0.430 sec)
INFO:tensorflow:global_step/sec: 245.674
INFO:tensorflow:step = 701, loss = 3.76325 (0.407 sec)
INFO:tensorflow:global_step/sec: 239.17
INFO:tensorflow:step = 801, loss = 4.85086 (0.418 sec)
INFO:tensorflow:global_step/sec: 235.764
INFO

In [318]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=10, num_epochs=1, shuffle=False)

In [319]:
results = model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2017-09-28-22:15:27
INFO:tensorflow:Restoring parameters from C:\Users\Green\AppData\Local\Temp\tmpnritn_33\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-09-28-22:15:31
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.524854, average_loss = 2.32314, global_step = 1000, loss = 23.2294


In [320]:
results

{'accuracy': 0.52485442,
 'average_loss': 2.323139,
 'global_step': 1000,
 'loss': 23.229429}

In [321]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size=10,num_epochs=1, shuffle=False)

In [322]:
predictions = model.predict(pred_input_func)

In [353]:
#list(predictions)

## We must go deeper

In [294]:
#for catagory columns
size = [3,66,11]

emb_Nominal = []
for j in range(len(Nominal)):
    emb_Nominal.append(tf.feature_column.embedding_column(Nominal[j], dimension=size[j]))

emb_Nominal    
#embedded_service = tf.feature_column.embedding_column(service, dimension=66)
#embedded_flag = tf.feature_column.embedding_column(flag, dimension=11)

#embed_nominal = [embedded_pt,embedded_service,embedded_flag ]

[_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='protocol_type', hash_bucket_size=3, dtype=tf.string), dimension=3, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x0000012AC9ADB588>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='service', hash_bucket_size=66, dtype=tf.string), dimension=66, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x0000012AC9ADB0F0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='flag', hash_bucket_size=11, dtype=tf.string), dimension=11, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x0000012AC9ADBE48>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)]

In [295]:
feat_cols = emb_Nominal + Numeric

In [325]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train, y_train, batch_size=10, num_epochs=1000, shuffle=True)

In [338]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[40,100,10], feature_columns=feat_cols, n_classes=6)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\Green\\AppData\\Local\\Temp\\tmpp14cy7u7', '_session_config': None}


In [344]:
dnn_model.train(input_fn=input_func, steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\Green\AppData\Local\Temp\tmpp14cy7u7\model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into C:\Users\Green\AppData\Local\Temp\tmpp14cy7u7\model.ckpt.
INFO:tensorflow:step = 1001, loss = 0.120365
INFO:tensorflow:global_step/sec: 312.654
INFO:tensorflow:step = 1101, loss = 0.117332 (0.324 sec)
INFO:tensorflow:global_step/sec: 317.62
INFO:tensorflow:step = 1201, loss = 0.163696 (0.315 sec)
INFO:tensorflow:global_step/sec: 340.39
INFO:tensorflow:step = 1301, loss = 0.0819727 (0.291 sec)
INFO:tensorflow:global_step/sec: 308.779
INFO:tensorflow:step = 1401, loss = 0.18231 (0.326 sec)
INFO:tensorflow:global_step/sec: 361.39
INFO:tensorflow:step = 1501, loss = 0.232271 (0.278 sec)
INFO:tensorflow:global_step/sec: 342.731
INFO:tensorflow:step = 1601, loss = 0.0479929 (0.291 sec)
INFO:tensorflow:global_step/sec: 349.904
INFO:tensorflow:step = 1701, loss = 0.277765 (0.287 sec)
INFO:tensorflow:

INFO:tensorflow:step = 9101, loss = 0.00384744 (0.289 sec)
INFO:tensorflow:global_step/sec: 372.15
INFO:tensorflow:step = 9201, loss = 0.0676948 (0.266 sec)
INFO:tensorflow:global_step/sec: 331.341
INFO:tensorflow:step = 9301, loss = 0.00133501 (0.303 sec)
INFO:tensorflow:global_step/sec: 325.928
INFO:tensorflow:step = 9401, loss = 0.0560638 (0.306 sec)
INFO:tensorflow:global_step/sec: 346.298
INFO:tensorflow:step = 9501, loss = 0.108389 (0.289 sec)
INFO:tensorflow:global_step/sec: 356.188
INFO:tensorflow:step = 9601, loss = 0.00830353 (0.282 sec)
INFO:tensorflow:global_step/sec: 362.714
INFO:tensorflow:step = 9701, loss = 0.0985334 (0.275 sec)
INFO:tensorflow:global_step/sec: 362.635
INFO:tensorflow:step = 9801, loss = 0.832755 (0.277 sec)
INFO:tensorflow:global_step/sec: 349.942
INFO:tensorflow:step = 9901, loss = 0.0597359 (0.283 sec)
INFO:tensorflow:global_step/sec: 346.301
INFO:tensorflow:step = 10001, loss = 0.0106158 (0.291 sec)
INFO:tensorflow:global_step/sec: 312.643
INFO:tens

# Train Data

In [345]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y=y_train, batch_size=100, num_epochs=1, shuffle=False)

In [346]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2017-09-28-22:24:08
INFO:tensorflow:Restoring parameters from C:\Users\Green\AppData\Local\Temp\tmpp14cy7u7\model.ckpt-11000
INFO:tensorflow:Finished evaluation at 2017-09-28-22:24:14
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.988783, average_loss = 0.0322011, global_step = 11000, loss = 3.21939


{'accuracy': 0.98878324,
 'average_loss': 0.032201067,
 'global_step': 11000,
 'loss': 3.2193909}

# Test data

In [347]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x = X_test, y=y_test, batch_size=100, num_epochs=1, shuffle=False)

In [348]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2017-09-28-22:24:15
INFO:tensorflow:Restoring parameters from C:\Users\Green\AppData\Local\Temp\tmpp14cy7u7\model.ckpt-11000
INFO:tensorflow:Finished evaluation at 2017-09-28-22:24:16
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.556503, average_loss = 2.73515, global_step = 11000, loss = 272.343


{'accuracy': 0.55650264,
 'average_loss': 2.7351542,
 'global_step': 11000,
 'loss': 272.3432}

### bad, lets try other models